In [132]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from sklearn.model_selection import train_test_split

In [134]:
#Load training dataset
df_train=pd.read_csv(r"D:\ML\DeepLearning\NLP_Project\train.csv")
df_test=pd.read_csv(r"D:\ML\DeepLearning\NLP_Project\test.csv")

In [136]:
#Display the first 5 rows of the data
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [138]:
# Split the data to text and labels
train_texts=df_train['text']
train_labels=df_train['target']
test_texts = df_test['text']

In [140]:
#Preprocess of text
tokenizer=Tokenizer(num_words=10000) #Only remain the high frequency word
tokenizer.fit_on_texts(train_texts)
train_sequences=tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [141]:
maxlen=100
X_train= pad_sequences(train_sequences, maxlen=maxlen)
y_train=train_labels.values
X_test = pad_sequences(test_sequences, maxlen=maxlen)

In [144]:
#Build a LSTM model
model=Sequential([
    Embedding(input_dim=10000,output_dim=128),
    LSTM(64,return_sequences=False),
    Dropout(0.5),
    Dense(1,activation="sigmoid")
])
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [146]:
#Train the model
model.fit(X_train,y_train,epochs=10,batch_size=32,validation_split=0.1)

Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.6456 - loss: 0.6169 - val_accuracy: 0.7979 - val_loss: 0.4406
Epoch 2/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.8649 - loss: 0.3383 - val_accuracy: 0.8123 - val_loss: 0.4250
Epoch 3/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9137 - loss: 0.2417 - val_accuracy: 0.8058 - val_loss: 0.4860
Epoch 4/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9437 - loss: 0.1687 - val_accuracy: 0.7927 - val_loss: 0.5809
Epoch 5/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9559 - loss: 0.1253 - val_accuracy: 0.7900 - val_loss: 0.6947
Epoch 6/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9641 - loss: 0.0961 - val_accuracy: 0.7848 - val_loss: 0.7527
Epoch 7/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9700 - loss: 0.0741 - val_accuracy: 0.7874 - val_loss: 0.7338
Epoch 8/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9710 - loss: 0.0672 - val_accu

In [148]:
#Predict the training dataset
y_pred=(model.predict(X_test)>0.5).astype("int32")

102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [152]:
df_submission=pd.read_csv(r"D:\ML\DeepLearning\NLP_Project\sample_submission.csv")
df_submission['target']=y_pred

In [156]:
df_submission.to_csv(r"D:\ML\DeepLearning\NLP_Project\submission.csv",index=False)
